In [1]:
!pip install -q streamlit transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [9]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
    model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Ensure pad_token_id is set to eos_token_id if pad_token_id is None
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return tokenizer, model, device

In [10]:
def generate_code(prompt, tokenizer, model, device, max_length=128, temperature=0.7, top_p=0.95):
    # Encode the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs.get("attention_mask", None).to(device)

    # Generate code using the model
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Pass attention mask
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,  # Use pad token id
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode and return the generated code
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code[len(prompt):]


In [12]:
def main():
    tokenizer, model, device = load_model()

    # Prompt input from the user
    prompt = input("Enter your prompt here: ")

    if prompt.strip() == "":
        print("Please enter a prompt.")
    else:
        print("Generating code...")
        generated_code = generate_code(prompt, tokenizer, model, device)
        print("Generated Code:")
        print(generated_code)

if __name__ == "__main__":
    main()

Enter your prompt here: Write a Python function to check if a number is prime.
Generating code...
Generated Code:

#
# def prime(n):
#     if n == 1:
#         return False
#     if n == 2:
#         return True
#     if n % 2 == 0:
#         return False
#     for i in range(3, n, 2):
#         if n % i == 0:
#             return False
#     return True
#
#
# def isPrime(n):
#     if n == 2 or n == 3:
#         return True
#     
